In [0]:
import torch
import torchvision
from torchvision import datasets, models, transforms
import os
from torch.autograd import Variable
import matplotlib.pyplot as plt
import time
from sklearn.metrics import f1_score
%matplotlib inline

In [2]:
!pip install efficientnet_pytorch
from efficientnet_pytorch import EfficientNet

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.6.3-cp36-none-any.whl size=12422 sha256=48404988ae378dca8e256f298a054686ff5dfcfab4183964c80b68a9c9518f6a
  Stored in directory: /root/.cache/pip/wheels/42/1e/a9/2a578ba9ad04e776e80bf0f70d8a7f4c29ec0718b92d8f6ccd
Successfully built efficientnet-pytorch


In [3]:
model=EfficientNet.from_pretrained('efficientnet-b5')
print(model)

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b5-b6417697.pth" to /root/.cache/torch/checkpoints/efficientnet-b5-b6417697.pth



Loaded pretrained weights for efficientnet-b5
EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 48, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_bn0): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        48, 48, kernel_size=(3, 3), stride=[1, 1], groups=48, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        48, 12, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        12, 48, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv

In [4]:
data_transform = transforms.Compose([transforms.Scale([224,224]),
                   transforms.Resize([224,224]),
                   transforms.CenterCrop([200,200]),
                   transforms.ToTensor(),
                   transforms.Normalize(mean=[0.5,0.5,0.5], std=[0.5,0.5,0.5])])

image_datasets = datasets.ImageFolder(root ="/content/drive/My Drive/game_gauzeMask_data_20200302.zip (Unzipped Files)/game_gauzeMask_data/train",
                                         transform = data_transform)
N=len(image_datasets)
print(N)
batch_size=20
import numpy as np
train_data,val_data=torch.utils.data.random_split(image_datasets,[600,69])
train_loader=torch.utils.data.DataLoader(train_data,batch_size=batch_size,shuffle=True)
val_loader=torch.utils.data.DataLoader(val_data,batch_size=batch_size,shuffle=True)

/usr/local/lib/python3.6/dist-packages/torchvision/transforms/transforms.py:220: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")


669


In [5]:
X_example, y_example = next(iter(train_loader))
example_clasees = image_datasets.classes
index_classes = image_datasets.class_to_idx
print(example_clasees)
print(index_classes)

['neg', 'pos']
{'neg': 0, 'pos': 1}


In [0]:
#model = models.googlenet(pretrained=True)
Use_gpu = torch.cuda.is_available()
device = torch.device("cuda:0" if torch.cuda.is_available else "cpu")
#print(model)

In [7]:
for parma in model.parameters():
    parma.requires_grad = False
    model._fc = torch.nn.Sequential(torch.nn.Linear(2048, 2))
if Use_gpu:
    model = model.cuda()
model.to(device)

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 48, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_bn0): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        48, 48, kernel_size=(3, 3), stride=[1, 1], groups=48, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        48, 12, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        12, 48, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        48, 24, kernel_siz

In [8]:
''' for parma in model.parameters():
    parma.requires_grad = False
    model.classifier = torch.nn.Sequential(torch.nn.Linear(1920, 960),
                                           torch.nn.ReLU(),
                                           torch.nn.Dropout(p=0.5),
                                           torch.nn.Linear(960, 480),
                                           torch.nn.ReLU(),
                                           torch.nn.Dropout(p=0.5),
                                           torch.nn.Linear(480, 2))
if Use_gpu:
    model = model.cuda()
model.to(device) '''

' for parma in model.parameters():\n    parma.requires_grad = False\n    model.classifier = torch.nn.Sequential(torch.nn.Linear(1920, 960),\n                                           torch.nn.ReLU(),\n                                           torch.nn.Dropout(p=0.5),\n                                           torch.nn.Linear(960, 480),\n                                           torch.nn.ReLU(),\n                                           torch.nn.Dropout(p=0.5),\n                                           torch.nn.Linear(480, 2))\nif Use_gpu:\n    model = model.cuda()\nmodel.to(device) '

In [0]:
loss_f = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model._fc.parameters(), lr = 0.0001)
epoch_n = 101

In [0]:
dataloader={
    "train":train_loader,
    "valid":val_loader
}

In [11]:
a=torch.tensor(1)
a=a.to(device)
a
b=torch.tensor([1,1,1,0,1]).to(device)
c=torch.tensor([1,0,1,0,0]).to(device)
e=torch.empty(len(b),1)
for i in range(len(b)):
  e[i]=(b[i]==c[i]==a)
torch.sum(e)

tensor(2.)

In [0]:
def train_model(dataloader,model,loss_f,optimizer,epoch_n):
  time_open = time.time()
  for epoch in range(epoch_n):
      print("Epoch {}/{}".format(epoch, epoch_n - 1))
      print("-"*10)
      
      for phase in ["train", "valid"]:
          if phase == "train":
              print("Training...")
              model.train(True)

          else:
              print("Validing...")
              model.train(False)

          running_loss = 0.0
          running_corrects = 0
          tp=0
          running_pos=0
          true_pos=0
      
          for batch, data in enumerate(dataloader[phase], 1):
              X, y = data
              E=torch.empty(len(y),1).to(device)
              O=torch.ones(len(y),1).to(device)
              X, y = Variable(X.cuda()), Variable(y.cuda())
              X = X.to(device)
              y = y.to(device)
          
              y_pred = model(X)
              _, pred = torch.max(y_pred.data, 1)
              optimizer.zero_grad()
              loss = loss_f(y_pred, y)
          
              if phase == "train":
                  loss.backward()
                  optimizer.step()

              for i in range(len(y)):
                E[i]=(pred[i]==y.data[i]==a)
              running_loss += loss.data
              running_corrects += torch.sum(pred == y.data)
              tp+=torch.sum(E)
              running_pos+=torch.sum(pred==O)
              true_pos+=torch.sum(y.data==O)
              

                
              if batch%50 == 0 and phase =="train":
                  print("Batch {}, Train Loss:{:.4f}, Train ACC:{:.4f}".format(batch, running_loss/batch, 100*running_corrects/(20*batch)))

          if phase=="train":
              epoch_loss = running_loss*20/600
              epoch_acc = 100*running_corrects/600
              
              P=tp/running_pos
              R=tp/true_pos
              print(tp,P,R)
              F1=2*P*R/(P+R)
              print("{} Loss:{:.4f} Acc:{:.4f}% F1:{:.2f}".format(phase, epoch_loss,epoch_acc,F1))
          else:
              epoch_loss = running_loss*20/69
              epoch_acc = 100*running_corrects/69
              P=tp/running_pos
              R=tp/true_pos
              print(tp,P,R)
              F1=2*P*R/(P+R)
              print("{} Loss:{:.4f} Acc:{:.4f}% F1:{:.2f}".format(phase, epoch_loss,epoch_acc,F1))
            
  time_end = time.time() - time_open
  print(time_end)
  return model

In [13]:
model=train_model(dataloader,model,loss_f,optimizer,epoch_n)

Epoch 0/100
----------
Training...


KeyboardInterrupt: ignored

In [0]:
test_datasets = datasets.ImageFolder(root ="/content/drive/My Drive/game_gauzeMask_data_20200302.zip (Unzipped Files)/game_gauzeMask_data/test",
                                         transform = data_transform)
print(len(test_datasets))
n=len(test_datasets)
test_loader=torch.utils.data.DataLoader(test_datasets,batch_size=100,shuffle=False)

In [0]:
example_clasees = test_datasets.classes
index_classes = test_datasets.class_to_idx
print(example_clasees)
print(index_classes)

In [0]:
TEST=torch.empty(39,100,dtype=int)
prediction=[]

In [0]:

for i, data in enumerate(test_loader):
  X, y = data
  X, y = Variable(X.cuda()), Variable(y.cuda())
  X = X.to(device)
  y = y.to(device)
  y_pred = model(X)
  _, pred = torch.max(y_pred.data, 1)
  predict=pred.data.cpu().numpy()
  print(predict)
  prediction.append(predict)


In [0]:
import pandas as pd

In [0]:
true=[]

In [0]:
for i in range(len(prediction)):
  sss=prediction[i]
  for s in sss:
    if s==1:
      s='pos'
      true.append(s)
    else:
      s='neg'
      true.append(s)

In [0]:
len(true)

In [0]:
true=np.array(true)

In [0]:
true.shape

In [0]:
X=pd.DataFrame(true,columns=["Label"])
X.to_csv("/content/drive/My Drive/X.csv")